# Chatbot

Use LLM to build a custom chatbot with only a modest amount of effort.

*ChatGPT* is a way for you to have a conversational interface, a conversation via a LLM.

You can use LLM to build your custom chatbot.

In [ ]:
import os
import openai
import panel

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = creds.api_key

## Functions

A function is created in which the specified prompt will be executed.

In [2]:
def get_completion(promtp, model = "gpt-3.5-turbo", temperature = 0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message["content"]

## The chat format

Chat models like *ChatGPT* are actually trained to take a series of messages as input and return a model generated message as output.

Chat model is trained to take a series of messages as input and then return a model generated message as output. So the user message is the input and then the assistant message is the output.

In [15]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    #print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [11]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

{
  "content": "Why, to reach the other side, kind sir! It seems that even chickens have the desire to explore new lands and seek new adventures.",
  "role": "assistant"
}
Why, to reach the other side, kind sir! It seems that even chickens have the desire to explore new lands and seek new adventures.


In [24]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':"Hi, my name is Jhonathan, I'm from Colombia"}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Jhonathan! It's nice to meet you. How can I assist you today?


Each conversation with a LLM is a standalone interaction, which means that you must provide all relevant messages for the model to dra from in the current conversation.

In [26]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes, can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as a chatbot, I don't have access to personal information. Therefore, I do not know your name.


If you want the model to draw from, or quote unquote remember earlier parts of a conversation, you must provide the earlier exchanges in the input to the model. And so we'll reter to this as context.

In [28]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':"Hi, my name is Jhonathan, I'm from Colombia"},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name? and How old Am I?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Jhonathan and as a chatbot, I don't have access to personal information like your age.


## OrderBot

We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant.

In [29]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard